<a href="https://colab.research.google.com/github/yecatstevir/teambrainiac/blob/main/source/AccuracyMeasures.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')#, force_remount = True)

Mounted at /content/gdrive


In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [3]:
# Clone the entire repo.
!git clone -l -s https://github.com/yecatstevir/teambrainiac.git
# Change directory into cloned repo
%cd teambrainiac/source
!ls

Cloning into 'teambrainiac'...
remote: Enumerating objects: 507, done.
remote: Counting objects: 100% (507/507), done.
remote: Compressing objects: 100% (348/348), done.
remote: Total 507 (delta 302), reused 302 (delta 145), pack-reused 0
Receiving objects: 100% (507/507), 60.60 MiB | 13.85 MiB/s, done.
Resolving deltas: 100% (302/302), done.
/content/teambrainiac/source
Access_Load_Data.ipynb		  Mat_to_Numpy.ipynb
All_subject_masked_labeled.ipynb  models
cross_validation.py		  percent_signal_change.ipynb
data				  process.py
Explore_data.ipynb		  SingleSubjectSVM.ipynb
explore.py			  SVM_Group_Child_Whole_Brain.ipynb
Images				  SVM_Group_YA_Whole_brain.ipynb
__init__.py			  utils.py
Masking.ipynb			  Visualize_Data.ipynb


In [4]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving path_config.py to path_config.py
User uploaded file "path_config.py" with length 228 bytes


In [5]:
# Import libraries
!pip install boto3 nilearn nibabel
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
import pickle
from utils import data_to_nib, load_mat, open_pickle, access_load_data, save_data, create_mask, labels_mask_binary, masking_data, masked_data_n_labels
from sklearn.svm import SVC
import numpy as np
import pandas as pd

     |████████████████████████████████| 132 kB 4.3 MB/s 
     |████████████████████████████████| 10.1 MB 31.2 MB/s 
     |████████████████████████████████| 8.6 MB 49.5 MB/s 
     |████████████████████████████████| 79 kB 10.0 MB/s 
     |████████████████████████████████| 138 kB 82.0 MB/s 
     |████████████████████████████████| 127 kB 85.3 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [58]:
path = "/content/drive/My Drive/data/model_dict_one_run_train.pkl"
data = open_pickle(path)

In [7]:
path = "data/data_path_dictionary.pkl"
data_path_dict = open_pickle(path)
subject_ids = data_path_dict['subject_ID']

In [35]:
file_name_paths = []
path = "/content/drive/My Drive/data/model_two_run_individual_models/"
def create_pickle_path(sub_id,path,file_beginning):
    destination_path = f"{path}{file_beginning}{sub_id}.pkl"
    return destination_path

In [44]:
def get_accuracy_scores(path,file_beginning):
  subj_acc = []
  for subj in subject_ids:
    #subj_data = model_dict[subj]
    destination_path = f"{path}{file_beginning}{subj}.pkl"
    model_dict = open_pickle(destination_path)
    clf = model_dict[subj]['model']
    X_train = model_dict[subj]['X_train']
    y_train = model_dict[subj]['y_train']
    X_test = model_dict[subj]['X_test'] 
    y_test = model_dict[subj]['y_test']
    if 'X_val' in model_dict[subj].keys():
      X_val = model_dict[subj]['X_val']
      y_val = model_dict[subj]['y_val']
      y_val_predicts = clf.predict(X_val)
      val_acc = accuracy_score(y_val,y_val_predicts)
    else:
      y_val_predicts = 0
      val_acc = 0
    train_predicts = clf.predict(X_train) 
    y_test_predicts = clf.predict(X_test)
    tr_acc = accuracy_score(y_train,train_predicts)   
    test_acc = accuracy_score(y_test,y_test_predicts)
    subj_list = [subj,tr_acc,val_acc,test_acc,'RUNS','PSCNORM']
    subj_acc.append(subj_list)
  return subj_acc

In [78]:
sub_acc_pd = get_accuracy_scores("/content/drive/My Drive/data/model_two_run_individual_models/",'sub_id_two_run_model_')


In [79]:
sub_acc_df = pd.DataFrame(sub_acc_pd, columns = ["Subject","TrainAcc","ValAcc","TestAcc","NormBy","DataType"])
path = "/content/drive/My Drive/data/model_two_run_individual_models.csv"
sub_acc_df.to_csv(path)

In [77]:
sub_model_one_run_psc_only_tuned

,Subject,TrainAcc,ValAcc,TestAcc,NormBy,DataType
0,10004_08693,1.0,0.607143,0.607143,RUNS,PSCNORM
1,10008_09924,1.0,0.666667,0.511905,RUNS,PSCNORM
2,10009_08848,1.0,0.702381,0.535714,RUNS,PSCNORM
3,10016_09694,1.0,0.726190,0.595238,RUNS,PSCNORM
4,10017_08894,1.0,0.630952,0.809524,RUNS,PSCNORM
5,10018_08907,1.0,0.654762,0.523810,RUNS,PSCNORM
6,10021_08839,1.0,0.571429,0.571429,RUNS,PSCNORM
7,10022_08854,1.0,0.464286,0.678571,RUNS,PSCNORM
8,10023_09126,1.0,0.488095,0.535714,RUNS,PSCNORM
9,10027_09455,1.0,0.571429,0.547619,RUNS,PSCNORM


In [82]:
path_sub = "/content/drive/My Drive/data/model_two_run_individual_models/sub_id_two_run_model_10004_08693.pkl"
data = open_pickle(path_sub)

In [83]:
data['10004_08693']['X_train']

array([[ 0.        ,  0.        ,  0.        , ...,  0.30043727,
        -0.18423449, -0.84468937],
       [ 0.        ,  0.        ,  0.        , ...,  0.69354737,
         1.3736299 ,  1.5898664 ],
       [ 0.        ,  0.        ,  0.        , ...,  0.81223184,
         1.2880856 ,  1.1930964 ],
       ...,
       [ 0.        ,  0.        ,  0.        , ...,  0.8612463 ,
         2.2765453 ,  1.1019093 ],
       [ 0.        ,  0.        ,  0.        , ..., -0.4619467 ,
        -0.35902548,  0.07365018],
       [ 0.        ,  0.        ,  0.        , ..., -0.46022683,
         0.14057651, -0.23217447]], dtype=float32)

In [92]:
data['10004_08693']['X_train'].shape

(168, 237979)

In [91]:
data['10004_08693']['X_test'][2]

array([ 0.       ,  0.       ,  0.       , ..., -1.6016942, -1.3019204,
       -1.3410923], dtype=float32)